In [2]:
import pandas as pd
from os import listdir
from os.path import isfile, join
import numpy as np
import datetime
from datetime import timedelta
from ta import add_all_ta_features
from ta.utils import dropna
from data_process_helpers import read_files, find_common_time_slice

In [3]:
# get data in desirved format
dow_files_addr = "./Data/Dow_30_1_min/"
etf_files_addr = "./Data/50_ETFs_1min/"

dow_files_names = [f for f in listdir(dow_files_addr)]
etf_files_names = [f for f in listdir(etf_files_addr)]

dow_names = [f.split(".")[0] for f in dow_files_names]
etf_names = [f.split(".")[0] for f in etf_files_names]

headers = ["date", "time", "open_price", "highest_price", "lowest_price", "close_price", "volumn"]
dtypes={"date": "str", 
    "time":"str",
    "open_price": "float", 
    "highest_price": "float",
    "lowest_price": "float", 
    "close_price": "float",
    "volumn": "int"}

print("Start read dow file...")
dow_dfs = read_files(dow_files_names, dow_files_addr, headers, dtypes)
print("Finish read dow file.")
print("---------------------")
print("Start read etf file...")
etf_dfs = read_files(etf_files_names, etf_files_addr, headers, dtypes)
print("Finish read etf file.")

Start read dow file...
Finish read dow file.
---------------------
Start read etf file...
Finish read etf file.


In [4]:
# get start and end for both dow and etf that cove evey df

dow_time_slice = [min(dow_dfs[0]["datetime"]), max(dow_dfs[0]["datetime"])]
etf_time_slice = [min(etf_dfs[0]["datetime"]), max(etf_dfs[0]["datetime"])]

print("Start check dow date...")
dow_time_slice = find_common_time_slice(dow_dfs, dow_time_slice, dow_files_names)
print("Finish check dow date...")
print("---------------------")
print("Start check etf date...")
etf_time_slice = find_common_time_slice(etf_dfs, etf_time_slice, etf_files_names)
print("Finish check etf date...")

Start check dow date...
start GM.txt
2010-11-18 09:36:00
end GM.txt
2020-06-05 19:44:00
Finish check dow date...
---------------------
Start check etf date...
start EEM.txt
2003-04-14 09:32:00
end EEM.txt
2020-06-05 19:41:00
end EFA.txt
2020-06-05 17:38:00
start ERX.txt
2008-11-19 09:30:00
start UPRO.txt
2009-06-25 09:40:00
Finish check etf date...


In [5]:
# limit all of the df to the same time frames
for i in range(len(dow_dfs)):
    dow_dfs[i] = dow_dfs[i].loc[(dow_dfs[i]["datetime"] >= dow_time_slice[0]) & (dow_dfs[i]["datetime"] <= dow_time_slice[1])]
for i in range(len(etf_dfs)):
    etf_dfs[i] = etf_dfs[i].loc[(etf_dfs[i]["datetime"] >= etf_time_slice[0]) & (etf_dfs[i]["datetime"] <= etf_time_slice[1])]

In [ ]:
dow_5min_dfs = modify_time_inteval(dow_dfs, dow_time_slice, 5)
etf_5min_dfs = modify_time_inteval(etf_dfs, etf_time_slice, 5)

In [138]:
def modify_time_inteval(dfs, time_slice, time_inteval):
    for i in range(len(dfs)):
        cur_open_price = dfs[i].iloc[0]["open_price"]
        cur_close_price = dfs[i].iloc[0]["close_price"]
        cur_highest_price = dfs[i].iloc[0]["highest_price"]  
        cur_lowest_price = dfs[i].iloc[0]["lowest_price"]
        target_time =  dfs[i].iloc[0]["datetime"]
        cur_volumn = dfs[i].iloc[0]["volumn"]
        
        start_time = time_slice[0]
        end_time = time_slice[1]
        cur_time = (start_time + timedelta(days=1)).replace(hour=9, minute=35)
        prev_time = cur_time - timedelta(minutes=5)
        
        headers = ["open_price", "highest_price", "lowest_price", "close_price", "volumn", "datetime", "hour", "minute"]
        df = pd.DataFrame(columns=headers)
        ndfs = []
        ###############################
        print("df: ", i)
        ###############################
        row_num = dfs[i].shape[0]
        j = 0
        while cur_time < end_time:
            is_first = True
            have_updates = False
            prev_close_price = cur_close_price

            cur_highest_price = float("-inf")
            cur_lowest_price = float("inf")
            cur_volumn = 0
            
            num_rows = dfs[i].shape[0]

            while target_time < cur_time and j < row_num:
                target_time = dfs[i].iloc[j]["datetime"]
                j += 1
                k = j -  1 if j < num_rows else num_rows - 1
                # k = j
                if target_time < prev_time or target_time > cur_time: continue
                have_updates = True
                cur_open_price = dfs[i].iloc[k]["open_price"] if is_first else cur_open_price
                is_first = False
                cur_close_price = dfs[i].iloc[k]["close_price"]
                cur_highest_price = dfs[i].iloc[k]["highest_price"] if cur_highest_price < dfs[i].iloc[k]["highest_price"] else cur_highest_price
                cur_lowest_price = dfs[i].iloc[k]["lowest_price"] if cur_lowest_price > dfs[i].iloc[k]["lowest_price"] else cur_lowest_price
                cur_volumn += dfs[i].iloc[k]["volumn"]

            if have_updates:
                df= df.append(pd.DataFrame(np.array([[cur_open_price, cur_highest_price, cur_lowest_price, cur_close_price, cur_volumn,
                cur_time, cur_time.to_pydatetime().hour, cur_time.to_pydatetime().minute]]), columns=headers), ignore_index=True) 
            else:
                df= df.append(pd.DataFrame(np.array([[prev_close_price, prev_close_price, prev_close_price, prev_close_price, 0,
                cur_time, cur_time.to_pydatetime().hour, cur_time.to_pydatetime().minute]]),columns=headers), ignore_index=True)
            # update time
            cur_time += timedelta(minutes=5)
            if cur_time.to_pydatetime().hour >= 16 and cur_time.to_pydatetime().minute != 0:
                cur_time += timedelta(days=1)
                cur_time =  cur_time.replace(hour=9, minute=35)
            prev_time = cur_time - timedelta(minutes=5)
        ndfs.append(df)
    return ndfs

In [139]:
datetime.datetime.min

datetime.datetime(1, 1, 1, 0, 0)

In [131]:
tmp_time_slice = []
tmp_time_slice.append((dow_time_slice[0]+ timedelta(days=4)).replace(hour=9, minute=35))
tmp_time_slice.append((dow_time_slice[0] + timedelta(days=10)).replace(hour=16, minute=0))

In [132]:
tmp_time_slice

[Timestamp('2010-11-22 09:35:00'), Timestamp('2010-11-28 16:00:00')]

In [133]:
test1_df

,open_price,highest_price,lowest_price,close_price,volumn,datetime,hour,minute
419363,29.22,29.27,29.22,29.25,38723,2010-11-23 09:36:00,9,36
419364,29.25,29.27,29.22,29.27,24611,2010-11-23 09:37:00,9,37
419365,29.27,29.31,29.25,29.27,54919,2010-11-23 09:38:00,9,38
419366,29.27,29.34,29.25,29.25,32370,2010-11-23 09:39:00,9,39
419367,29.27,29.29,29.22,29.27,43094,2010-11-23 09:40:00,9,40
...,...,...,...,...,...,...,...,...
420258,29.45,29.51,29.45,29.51,15820,2010-11-26 09:51:00,9,51
420259,29.49,29.51,29.43,29.45,10939,2010-11-26 09:52:00,9,52
420260,29.45,29.47,29.43,29.46,17487,2010-11-26 09:53:00,9,53
420261,29.45,29.47,29.45,29.47,3704,2010-11-26 09:54:00,9,54


In [134]:
38723+24611+54919+32370+43094

193717

In [140]:
a = modify_time_inteval([test1_df], tmp_time_slice, 5)

df:  0
cur_time:  2010-11-23 09:35:00
Write with volumn:  0
cur_time:  2010-11-23 09:40:00
target_time:  2010-11-23 09:36:00
j:  1 k:  0
volumn:  38723
target_time:  2010-11-23 09:37:00
j:  2 k:  1
volumn:  24611
target_time:  2010-11-23 09:38:00
j:  3 k:  2
volumn:  54919
target_time:  2010-11-23 09:39:00
j:  4 k:  3
volumn:  32370
target_time:  2010-11-23 09:40:00
j:  5 k:  4
volumn:  43094
Write with volumn:  193717
cur_time:  2010-11-23 09:45:00
target_time:  2010-11-23 09:41:00
j:  6 k:  5
volumn:  30307
target_time:  2010-11-23 09:42:00
j:  7 k:  6
volumn:  38125
target_time:  2010-11-23 09:43:00
j:  8 k:  7
volumn:  50168
target_time:  2010-11-23 09:44:00
j:  9 k:  8
volumn:  15148
target_time:  2010-11-23 09:45:00
j:  10 k:  9
volumn:  19228
Write with volumn:  152976
cur_time:  2010-11-23 09:50:00
target_time:  2010-11-23 09:46:00
j:  11 k:  10
volumn:  27050
target_time:  2010-11-23 09:47:00
j:  12 k:  11
volumn:  18766
target_time:  2010-11-23 09:48:00
j:  13 k:  12
volumn: 

cur_time:  2010-11-23 12:05:00
target_time:  2010-11-23 12:01:00
j:  146 k:  145
volumn:  4268
target_time:  2010-11-23 12:02:00
j:  147 k:  146
volumn:  17688
target_time:  2010-11-23 12:03:00
j:  148 k:  147
volumn:  15334
target_time:  2010-11-23 12:04:00
j:  149 k:  148
volumn:  30029
target_time:  2010-11-23 12:05:00
j:  150 k:  149
volumn:  26834
Write with volumn:  94153
cur_time:  2010-11-23 12:10:00
target_time:  2010-11-23 12:06:00
j:  151 k:  150
volumn:  3671
target_time:  2010-11-23 12:07:00
j:  152 k:  151
volumn:  16181
target_time:  2010-11-23 12:08:00
j:  153 k:  152
volumn:  14426
target_time:  2010-11-23 12:09:00
j:  154 k:  153
volumn:  1069
target_time:  2010-11-23 12:10:00
j:  155 k:  154
volumn:  24529
Write with volumn:  59876
cur_time:  2010-11-23 12:15:00
target_time:  2010-11-23 12:11:00
j:  156 k:  155
volumn:  1747
target_time:  2010-11-23 12:12:00
j:  157 k:  156
volumn:  6190
target_time:  2010-11-23 12:13:00
j:  158 k:  157
volumn:  3524
target_time:  20

cur_time:  2010-11-23 14:30:00
target_time:  2010-11-23 14:26:00
j:  291 k:  290
volumn:  40163
target_time:  2010-11-23 14:27:00
j:  292 k:  291
volumn:  22790
target_time:  2010-11-23 14:28:00
j:  293 k:  292
volumn:  3297
target_time:  2010-11-23 14:29:00
j:  294 k:  293
volumn:  3007
target_time:  2010-11-23 14:30:00
j:  295 k:  294
volumn:  20762
Write with volumn:  90019
cur_time:  2010-11-23 14:35:00
target_time:  2010-11-23 14:31:00
j:  296 k:  295
volumn:  26677
target_time:  2010-11-23 14:32:00
j:  297 k:  296
volumn:  19525
target_time:  2010-11-23 14:33:00
j:  298 k:  297
volumn:  3890
target_time:  2010-11-23 14:34:00
j:  299 k:  298
volumn:  9237
target_time:  2010-11-23 14:35:00
j:  300 k:  299
volumn:  5280
Write with volumn:  64609
cur_time:  2010-11-23 14:40:00
target_time:  2010-11-23 14:36:00
j:  301 k:  300
volumn:  1760
target_time:  2010-11-23 14:37:00
j:  302 k:  301
volumn:  16012
target_time:  2010-11-23 14:38:00
j:  303 k:  302
volumn:  1212
target_time:  201

j:  484 k:  483
volumn:  9522
target_time:  2010-11-24 10:19:00
j:  485 k:  484
volumn:  6220
target_time:  2010-11-24 10:20:00
j:  486 k:  485
volumn:  31406
Write with volumn:  60943
cur_time:  2010-11-24 10:25:00
target_time:  2010-11-24 10:21:00
j:  487 k:  486
volumn:  22822
target_time:  2010-11-24 10:22:00
j:  488 k:  487
volumn:  35007
target_time:  2010-11-24 10:23:00
j:  489 k:  488
volumn:  25707
target_time:  2010-11-24 10:24:00
j:  490 k:  489
volumn:  4473
target_time:  2010-11-24 10:25:00
j:  491 k:  490
volumn:  34434
Write with volumn:  122443
cur_time:  2010-11-24 10:30:00
target_time:  2010-11-24 10:26:00
j:  492 k:  491
volumn:  5955
target_time:  2010-11-24 10:27:00
j:  493 k:  492
volumn:  29032
target_time:  2010-11-24 10:28:00
j:  494 k:  493
volumn:  8490
target_time:  2010-11-24 10:29:00
j:  495 k:  494
volumn:  6032
target_time:  2010-11-24 10:30:00
j:  496 k:  495
volumn:  30240
Write with volumn:  79749
cur_time:  2010-11-24 10:35:00
target_time:  2010-11-2

cur_time:  2010-11-24 12:50:00
target_time:  2010-11-24 12:46:00
j:  632 k:  631
volumn:  3057
target_time:  2010-11-24 12:47:00
j:  633 k:  632
volumn:  24758
target_time:  2010-11-24 12:48:00
j:  634 k:  633
volumn:  1469
target_time:  2010-11-24 12:49:00
j:  635 k:  634
volumn:  1907
target_time:  2010-11-24 12:50:00
j:  636 k:  635
volumn:  1491
Write with volumn:  32682
cur_time:  2010-11-24 12:55:00
target_time:  2010-11-24 12:51:00
j:  637 k:  636
volumn:  2223
target_time:  2010-11-24 12:52:00
j:  638 k:  637
volumn:  2115
target_time:  2010-11-24 12:53:00
j:  639 k:  638
volumn:  4047
target_time:  2010-11-24 12:54:00
j:  640 k:  639
volumn:  1625
target_time:  2010-11-24 12:55:00
j:  641 k:  640
volumn:  16356
Write with volumn:  26366
cur_time:  2010-11-24 13:00:00
target_time:  2010-11-24 12:56:00
j:  642 k:  641
volumn:  3016
target_time:  2010-11-24 12:57:00
j:  643 k:  642
volumn:  1431
target_time:  2010-11-24 12:58:00
j:  644 k:  643
volumn:  2589
target_time:  2010-11

target_time:  2010-11-24 14:38:00
j:  744 k:  743
volumn:  23927
target_time:  2010-11-24 14:39:00
j:  745 k:  744
volumn:  200037
target_time:  2010-11-24 14:40:00
j:  746 k:  745
volumn:  77653
Write with volumn:  352903
cur_time:  2010-11-24 14:45:00
target_time:  2010-11-24 14:41:00
j:  747 k:  746
volumn:  195564
target_time:  2010-11-24 14:42:00
j:  748 k:  747
volumn:  96553
target_time:  2010-11-24 14:43:00
j:  749 k:  748
volumn:  146002
target_time:  2010-11-24 14:44:00
j:  750 k:  749
volumn:  1247
target_time:  2010-11-24 14:45:00
j:  751 k:  750
volumn:  1647
Write with volumn:  441013
cur_time:  2010-11-24 14:50:00
target_time:  2010-11-24 14:46:00
j:  752 k:  751
volumn:  1202
target_time:  2010-11-24 14:47:00
j:  753 k:  752
volumn:  5209
target_time:  2010-11-24 14:48:00
j:  754 k:  753
volumn:  3721
target_time:  2010-11-24 14:49:00
j:  755 k:  754
volumn:  1844
target_time:  2010-11-24 14:50:00
j:  756 k:  755
volumn:  1224
Write with volumn:  13200
cur_time:  2010-1

cur_time:  2010-11-25 15:45:00
Write with volumn:  0
cur_time:  2010-11-25 15:50:00
Write with volumn:  0
cur_time:  2010-11-25 15:55:00
Write with volumn:  0
cur_time:  2010-11-25 16:00:00
Write with volumn:  0
cur_time:  2010-11-26 09:35:00
target_time:  2010-11-26 04:46:00
target_time:  2010-11-26 07:30:00
target_time:  2010-11-26 07:56:00
target_time:  2010-11-26 08:00:00
target_time:  2010-11-26 08:05:00
target_time:  2010-11-26 08:06:00
target_time:  2010-11-26 08:09:00
target_time:  2010-11-26 08:10:00
target_time:  2010-11-26 08:11:00
target_time:  2010-11-26 08:16:00
target_time:  2010-11-26 08:17:00
target_time:  2010-11-26 08:20:00
target_time:  2010-11-26 08:23:00
target_time:  2010-11-26 08:26:00
target_time:  2010-11-26 08:28:00
target_time:  2010-11-26 08:33:00
target_time:  2010-11-26 08:36:00
target_time:  2010-11-26 08:39:00
target_time:  2010-11-26 08:41:00
target_time:  2010-11-26 08:42:00
target_time:  2010-11-26 08:45:00
target_time:  2010-11-26 08:49:00
target_ti

cur_time:  2010-11-27 15:05:00
Write with volumn:  0
cur_time:  2010-11-27 15:10:00
Write with volumn:  0
cur_time:  2010-11-27 15:15:00
Write with volumn:  0
cur_time:  2010-11-27 15:20:00
Write with volumn:  0
cur_time:  2010-11-27 15:25:00
Write with volumn:  0
cur_time:  2010-11-27 15:30:00
Write with volumn:  0
cur_time:  2010-11-27 15:35:00
Write with volumn:  0
cur_time:  2010-11-27 15:40:00
Write with volumn:  0
cur_time:  2010-11-27 15:45:00
Write with volumn:  0
cur_time:  2010-11-27 15:50:00
Write with volumn:  0
cur_time:  2010-11-27 15:55:00
Write with volumn:  0
cur_time:  2010-11-27 16:00:00
Write with volumn:  0
cur_time:  2010-11-28 09:35:00
Write with volumn:  0
cur_time:  2010-11-28 09:40:00
Write with volumn:  0
cur_time:  2010-11-28 09:45:00
Write with volumn:  0
cur_time:  2010-11-28 09:50:00
Write with volumn:  0
cur_time:  2010-11-28 09:55:00
Write with volumn:  0
cur_time:  2010-11-28 10:00:00
Write with volumn:  0
cur_time:  2010-11-28 10:05:00
Write with volu

In [141]:
a[0]

,open_price,highest_price,lowest_price,close_price,volumn,datetime,hour,minute
0,29.25,29.25,29.25,29.25,0,2010-11-23 09:35:00,9,35
1,29.22,29.34,29.22,29.27,193717,2010-11-23 09:40:00,9,40
2,29.28,29.34,29.2,29.27,152976,2010-11-23 09:45:00,9,45
3,29.27,29.34,29.22,29.25,117510,2010-11-23 09:50:00,9,50
4,29.25,29.29,29.2,29.22,151294,2010-11-23 09:55:00,9,55
...,...,...,...,...,...,...,...,...
462,29.47,29.47,29.47,29.47,0,2010-11-28 15:35:00,15,35
463,29.47,29.47,29.47,29.47,0,2010-11-28 15:40:00,15,40
464,29.47,29.47,29.47,29.47,0,2010-11-28 15:45:00,15,45
465,29.47,29.47,29.47,29.47,0,2010-11-28 15:50:00,15,50


In [142]:
a[0]

,open_price,highest_price,lowest_price,close_price,volumn,datetime,hour,minute
0,29.25,29.25,29.25,29.25,0,2010-11-23 09:35:00,9,35
1,29.22,29.34,29.22,29.27,193717,2010-11-23 09:40:00,9,40
2,29.28,29.34,29.2,29.27,152976,2010-11-23 09:45:00,9,45
3,29.27,29.34,29.22,29.25,117510,2010-11-23 09:50:00,9,50
4,29.25,29.29,29.2,29.22,151294,2010-11-23 09:55:00,9,55
...,...,...,...,...,...,...,...,...
462,29.47,29.47,29.47,29.47,0,2010-11-28 15:35:00,15,35
463,29.47,29.47,29.47,29.47,0,2010-11-28 15:40:00,15,40
464,29.47,29.47,29.47,29.47,0,2010-11-28 15:45:00,15,45
465,29.47,29.47,29.47,29.47,0,2010-11-28 15:50:00,15,50


In [71]:
test1_df.iloc[839]["volumn"]

89

In [67]:
test1_df.shape

(900, 8)

In [68]:
test1_df.iloc[838]

open_price                     29.83
highest_price                  29.83
lowest_price                   29.83
close_price                    29.83
volumn                           134
datetime         2010-11-24 19:36:00
hour                              19
minute                            36
Name: 420201, dtype: object

In [69]:
test1_df.iloc[839]

open_price                     29.36
highest_price                  29.36
lowest_price                   29.36
close_price                    29.36
volumn                            89
datetime         2010-11-26 04:40:00
hour                               4
minute                            40
Name: 420202, dtype: object

In [47]:
a[0]

,open_price,highest_price,lowest_price,close_price,volumn,datetime,hour,minute
0,29.25,29.25,29.25,29.25,0,2010-11-23 09:35:00,9,35
1,29.22,29.34,29.22,29.27,193717,2010-11-23 09:40:00,9,40
2,29.28,29.34,29.2,29.27,152976,2010-11-23 09:45:00,9,45
3,29.27,29.34,29.22,29.25,117510,2010-11-23 09:50:00,9,50
4,29.25,29.29,29.2,29.22,151294,2010-11-23 09:55:00,9,55
...,...,...,...,...,...,...,...,...
462,29.47,29.47,29.47,29.47,0,2010-11-28 15:35:00,15,35
463,29.47,29.47,29.47,29.47,0,2010-11-28 15:40:00,15,40
464,29.47,29.47,29.47,29.47,0,2010-11-28 15:45:00,15,45
465,29.47,29.47,29.47,29.47,0,2010-11-28 15:50:00,15,50


In [143]:
a[0].to_csv("a.csv")

In [11]:
test1_df = dow_dfs[0].loc[dow_dfs[0]["datetime"] > tmp_time_slice[0]+timedelta(days=1)].iloc[0:900]

In [12]:
test1_df

,open_price,highest_price,lowest_price,close_price,volumn,datetime,hour,minute
419363,29.22,29.27,29.22,29.25,38723,2010-11-23 09:36:00,9,36
419364,29.25,29.27,29.22,29.27,24611,2010-11-23 09:37:00,9,37
419365,29.27,29.31,29.25,29.27,54919,2010-11-23 09:38:00,9,38
419366,29.27,29.34,29.25,29.25,32370,2010-11-23 09:39:00,9,39
419367,29.27,29.29,29.22,29.27,43094,2010-11-23 09:40:00,9,40
...,...,...,...,...,...,...,...,...
420258,29.45,29.51,29.45,29.51,15820,2010-11-26 09:51:00,9,51
420259,29.49,29.51,29.43,29.45,10939,2010-11-26 09:52:00,9,52
420260,29.45,29.47,29.43,29.46,17487,2010-11-26 09:53:00,9,53
420261,29.45,29.47,29.45,29.47,3704,2010-11-26 09:54:00,9,54


In [87]:
38723 + 24611 + 54919 + 32370 + 43094

193717

In [19]:
38125 + 50168 + 15148 + 19228 + 27050

149719

In [56]:
a = 1 if True else 2
print(a)

1


In [5]:
dow_time_slice

[Timestamp('2010-11-18 09:36:00'), Timestamp('2020-06-05 19:44:00')]

In [6]:
etf_time_slice

[Timestamp('2009-06-25 09:40:00'), Timestamp('2020-06-05 17:38:00')]

In [7]:
# limit all of the df to the same time frames
for i in range(len(dow_dfs)):
    dow_dfs[i] = dow_dfs[i].loc[(dow_dfs[i]["datetime"] >= dow_time_slice[0]) & (dow_dfs[i]["datetime"] <= dow_time_slice[1])]
for i in range(len(etf_dfs)):
    etf_dfs[i] = etf_dfs[i].loc[(etf_dfs[i]["datetime"] >= etf_time_slice[0]) & (etf_dfs[i]["datetime"] <= etf_time_slice[1])]

In [17]:
tmp_time_slice = []
tmp_time_slice.append((dow_time_slice[0]+ timedelta(days=4)).replace(hour=9, minute=35))
tmp_time_slice.append((dow_time_slice[0] + timedelta(days=6)).replace(hour=16, minute=0))

In [18]:
tmp_time_slice

[Timestamp('2010-11-22 09:30:00'), Timestamp('2010-11-24 16:00:00')]

In [25]:
dow_dfs[0]

,open_price,close_price,highest_price,lowest_price,volumn,datetime,hour,minute
418027,29.85,29.87,29.84,29.85,169113,2010-11-18 09:36:00,9,36
418028,29.85,29.92,29.84,29.87,41218,2010-11-18 09:37:00,9,37
418029,29.87,29.90,29.85,29.85,13885,2010-11-18 09:38:00,9,38
418030,29.87,29.90,29.81,29.83,39934,2010-11-18 09:39:00,9,39
418031,29.82,29.85,29.78,29.84,57570,2010-11-18 09:40:00,9,40
...,...,...,...,...,...,...,...,...
1470592,12.04,12.04,12.04,12.04,100,2020-06-05 18:59:00,18,59
1470593,12.04,12.04,12.04,12.04,209,2020-06-05 19:40:00,19,40
1470594,12.05,12.05,12.05,12.05,100,2020-06-05 19:42:00,19,42
1470595,12.05,12.05,12.05,12.05,1200,2020-06-05 19:43:00,19,43


In [47]:
a = modify_time_inteval([dow_dfs[0]], tmp_time_slice, 5)

df:  0
2010-11-23 09:30:00
2010-11-23 09:35:00
2010-11-23 09:40:00
2010-11-23 09:45:00
2010-11-23 09:50:00
2010-11-23 09:55:00
2010-11-23 10:00:00
2010-11-23 10:05:00
2010-11-23 10:10:00
2010-11-23 10:15:00
2010-11-23 10:20:00
2010-11-23 10:25:00
2010-11-23 10:30:00
2010-11-23 10:35:00
2010-11-23 10:40:00
2010-11-23 10:45:00
2010-11-23 10:50:00
2010-11-23 10:55:00
2010-11-23 11:00:00
2010-11-23 11:05:00
2010-11-23 11:10:00
2010-11-23 11:15:00
2010-11-23 11:20:00
2010-11-23 11:25:00
2010-11-23 11:30:00
2010-11-23 11:35:00
2010-11-23 11:40:00
2010-11-23 11:45:00
2010-11-23 11:50:00
2010-11-23 11:55:00
2010-11-23 12:00:00
2010-11-23 12:05:00
2010-11-23 12:10:00
2010-11-23 12:15:00
2010-11-23 12:20:00
2010-11-23 12:25:00
2010-11-23 12:30:00
2010-11-23 12:35:00
2010-11-23 12:40:00
2010-11-23 12:45:00
2010-11-23 12:50:00
2010-11-23 12:55:00
2010-11-23 13:00:00
2010-11-23 13:05:00
2010-11-23 13:10:00
2010-11-23 13:15:00
2010-11-23 13:20:00
2010-11-23 13:25:00
2010-11-23 13:30:00
2010-11-23 13

In [50]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(a[0])

    open_price close_price highest_price lowest_price    volumn  \
0        12.05       29.49         30.41        12.05  25267385   
1        29.41       29.22         29.29         29.2    294258   
2        29.22       29.29         29.25        29.25    204441   
3        29.28       29.31         29.27        29.25    157056   
4        29.27       29.34         29.29        29.25    130923   
5        29.25       29.25         29.22        29.22    150319   
6        29.22        29.2          29.2        29.15    270636   
7        29.16       29.29         29.25        29.22    187009   
8        29.27       29.31         29.28        29.25     91245   
9        29.28        29.2          29.2        29.16    182662   
10       29.18       29.25         29.25         29.2    100233   
11        29.2       29.22         29.18        29.17     87962   
12        29.2       29.16         29.16        29.13    192666   
13       29.16       29.18          29.2        29.18    14194

In [51]:
a[0]

,open_price,close_price,highest_price,lowest_price,volumn,datetime,hour,minute
0,12.05,29.49,30.41,12.05,25267385,2010-11-23 09:30:00,9,30
1,29.41,29.22,29.29,29.2,294258,2010-11-23 09:35:00,9,35
2,29.22,29.29,29.25,29.25,204441,2010-11-23 09:40:00,9,40
3,29.28,29.31,29.27,29.25,157056,2010-11-23 09:45:00,9,45
4,29.27,29.34,29.29,29.25,130923,2010-11-23 09:50:00,9,50
...,...,...,...,...,...,...,...,...
152,29.83,29.85,29.83,29.81,66630,2010-11-24 15:35:00,15,35
153,29.83,29.83,29.81,29.81,41296,2010-11-24 15:40:00,15,40
154,29.83,29.83,29.81,29.81,355331,2010-11-24 15:45:00,15,45
155,29.81,29.83,29.81,29.81,37018,2010-11-24 15:50:00,15,50


In [49]:
53678 + 60384 + 87462 + 101494 + 46367 - 294258

55127

In [41]:
dow_dfs[0].iloc[836]["datetime"]

Timestamp('2010-11-19 19:59:00')

In [40]:
dow_dfs[0]

,open_price,close_price,highest_price,lowest_price,volumn,datetime,hour,minute
418027,29.85,29.87,29.84,29.85,169113,2010-11-18 09:36:00,9,36
418028,29.85,29.92,29.84,29.87,41218,2010-11-18 09:37:00,9,37
418029,29.87,29.90,29.85,29.85,13885,2010-11-18 09:38:00,9,38
418030,29.87,29.90,29.81,29.83,39934,2010-11-18 09:39:00,9,39
418031,29.82,29.85,29.78,29.84,57570,2010-11-18 09:40:00,9,40
...,...,...,...,...,...,...,...,...
1470592,12.04,12.04,12.04,12.04,100,2020-06-05 18:59:00,18,59
1470593,12.04,12.04,12.04,12.04,209,2020-06-05 19:40:00,19,40
1470594,12.05,12.05,12.05,12.05,100,2020-06-05 19:42:00,19,42
1470595,12.05,12.05,12.05,12.05,1200,2020-06-05 19:43:00,19,43
